In [1]:
!rm -rf YOLOv8-Lightweight-Backbones
!git clone https://github.com/Brintsov/YOLOv8-Lightweight-Backbones

Cloning into 'YOLOv8-Lightweight-Backbones'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 122 (delta 27), reused 56 (delta 16), pack-reused 37 (from 1)
Receiving objects: 100% (122/122), 113.30 MiB | 25.64 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
import sys
sys.path.append('./YOLOv8-Lightweight-Backbones')

In [3]:
import numpy as np
from keras_cv import visualization
from data.prepate_data import run_data_preparation_pipeline
from models.csp_darknet import create_xs_csp_darknet_yolov8
from models.ghostnet_v2 import create_0_5_ghostnet_yolov8
from models.shufflenet_v2 import create_0_5_shufflenet_yolov8

from keras.callbacks import CSVLogger
from profiling.coco_metric_callback import EvaluateCOCOMetricsCallback
from profiling.profilers import ModelsProfiler, FlopsExtractor, InferenceProfiler, COCOMetricsCalculator
import warnings
warnings.filterwarnings("ignore")

num_classes = 2
input_shape = (640, 640, 3)


In [4]:
gathered_data = run_data_preparation_pipeline()
train_ds, val_ds = gathered_data['train_ds'], gathered_data['val_ds']
cat_mapping_r = gathered_data['cat_mapping_r']
cat_mapping = gathered_data['cat_mapping']

Path to dataset files: /root/.cache/kagglehub/datasets/trainingdatapro/cars-video-object-tracking/versions/3


In [5]:
model_shuffle = create_0_5_shufflenet_yolov8(num_classes=num_classes, input_shape=input_shape)
csp_darknet_yolo = create_xs_csp_darknet_yolov8(num_classes=num_classes)
ghost_model = create_0_5_ghostnet_yolov8(num_classes=num_classes, input_shape=input_shape)

csp_darknet_yolo.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_csp_darknet_yolo.keras")
model_shuffle.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_shuffle_yolo.keras")
ghost_model.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_ghost_yolo.keras")

In [6]:
example_data_for_inf = val_ds.take(1)
example_data_for_inf = next(iter(example_data_for_inf))[0][:1]

In [7]:
profilers = [
    InferenceProfiler(repeats=50, device='CPU:0'),
    InferenceProfiler(repeats=50, device='GPU:0'),
    FlopsExtractor(),
    COCOMetricsCalculator(val_ds)
]
models = {
    'ShuffleNetV2-YOLOv8': model_shuffle,
    'CSPDarkNet-YOLOv8': csp_darknet_yolo,
    'GhostNetv2-YOLOv8': ghost_model,
}
general_profiles = ModelsProfiler(profilers=profilers)
profile_results = general_profiles.profile(models, example_data_for_inf)

RUNNING  <profiling.profilers.InferenceProfiler object at 0x7b4b9414d910> FOR MODEL ShuffleNetV2-YOLOv8
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7b4b9414eb70> FOR MODEL ShuffleNetV2-YOLOv8
RUNNING  <profiling.profilers.FlopsExtractor object at 0x7b4b9414d400> FOR MODEL ShuffleNetV2-YOLOv8


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


RUNNING  <profiling.profilers.COCOMetricsCalculator object at 0x7b4b9414c260> FOR MODEL ShuffleNetV2-YOLOv8
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7b4b9414d910> FOR MODEL CSPDarkNet-YOLOv8
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7b4b9414eb70> FOR MODEL CSPDarkNet-YOLOv8
RUNNING  <profiling.profilers.FlopsExtractor object at 0x7b4b9414d400> FOR MODEL CSPDarkNet-YOLOv8
RUNNI

In [8]:
profile_results.columns

Index(['avg_latency_sec', 'median_latency_sec', 'p90_latency_sec',
       'min_latency_sec', 'max_latency_sec', 'std_latency_sec', 'CPU:0_fps',
       'ram_delta_mb', 'cpu_percent_before', 'cpu_percent_after',
       'cpu_user_time_delta_s', 'cpu_sys_time_delta_s', 'gpu_current_mem_mb',
       'gpu_peak_mem_mb', 'GPU:0_fps', 'flops', 'flops_g', 'params_m', 'MaP',
       'MaP@[IoU=50]', 'MaP@[IoU=75]', 'MaP@[area=small]', 'MaP@[area=medium]',
       'MaP@[area=large]', 'Recall@[max_detections=1]',
       'Recall@[max_detections=10]', 'Recall@[max_detections=100]',
       'Recall@[area=small]', 'Recall@[area=medium]', 'Recall@[area=large]',
       'model'],
      dtype='object')

In [10]:
profile_results[['flops_g', 'MaP', "CPU:0_fps", 'GPU:0_fps']]

,flops_g,MaP,CPU:0_fps,GPU:0_fps
0,8.590339,0.934341,1.409470,2.506548
1,9.724187,0.936953,1.587277,3.371189
2,2.231183,0.930949,1.738294,2.931556


In [11]:
from quantization.lite_wrapper import YOLOv8LiteWrapper


shuffle_lite_model = YOLOv8LiteWrapper("./YOLOv8-Lightweight-Backbones/quantized_models/shuffle.tflite")
ghost_lite_model = YOLOv8LiteWrapper("./YOLOv8-Lightweight-Backbones/quantized_models/ghost.tflite")
csp_lite_model = YOLOv8LiteWrapper("./YOLOv8-Lightweight-Backbones/quantized_models/csp.tflite")

ValueError: num_threads should >= 1